# Prompt_Tuning:冻结主模型全部参数，在训练数据前加入一小段Prompt

# 1.导包

In [18]:
from babel.plural import skip_token
from datasets import load_dataset,Dataset
from sympy.stats.sampling.sample_pymc import do_sample_pymc

from transformers import AutoTokenizer,AutoModelForCausalLM,DataCollatorForSeq2Seq,TrainingArguments,Trainer

# 2.加载数据集

In [19]:
ds = load_dataset("shibing624/alpaca-zh")
ds = ds["train"]
print(ds)

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 48818
})


In [20]:
ds[:10]

{'instruction': ['保持健康的三个提示。',
  '三原色是什么？',
  '描述原子的结构。',
  '如何减少空气污染？',
  '描述一次你不得不做出困难决定的经历。',
  '辨认不同之处。 ',
  '解释为什么以下分数等同于1/4',
  '写一个关于主人公必须做出重要职业决定的第三人称叙述的短故事。',
  '渲染一座房子的三维模型',
  '朱利叶斯·凯撒是如何死亡的？'],
 'input': ['',
  '',
  '',
  '',
  '',
  '输入：Twitter，Instagram，Telegram。',
  '输入：4/16',
  '',
  '',
  ''],
 'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。',
  '三原色通常指的是红色、绿色和蓝色（RGB）。它们是通过加色混合原理创建色彩的三种基础颜色。在以发光为基础的显示设备中（如电视、计算机显示器、智能手机和平板电脑显示屏）, 三原色可混合产生大量色彩。其中红色和绿色可以混合生成黄色，红色和蓝色可以混合生成品红色，蓝色和绿色可以混合生成青色。当红色、绿色和蓝色按相等比例混合时，可以产生白色或灰色。\n\n此外，在印刷和绘画中，三原色指的是以颜料为基础的红、黄和蓝颜色（RYB）。这三种颜色用以通过减色混合原理来创建色彩。不过，三原色的具体定义并不唯一，不同的颜色系统可能会采用不同的三原色。',
  '原子是物质的基本单位，它由三种基本粒子组成：质子、中子和电子。质子和中子形成原子核，位于原子中心，核外的电子围绕着原子核运动。\n\n原子结构具有层次性。原子核中，质子带正电，中子不带电（中性）。原子核非常小且致密，占据了原子总质量的绝大部分。电子带负电，通常围绕核运动，形成若干层次，称为壳层或电子层。电子数量与质子数量相等，使原子呈电中性。\n\n电子在每个

# 3.数据集预处理

In [4]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh")
tokenizer

BloomTokenizerFast(name_or_path='Langboat/bloom-1b4-zh', vocab_size=46145, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [5]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human:" + example["instruction"],example["input"]]).strip() + "\n\nAssistant: ")
    # print(instruction)
    response = tokenizer(example["output"] + tokenizer.eos_token)
    # print( response)
    input_ids = instruction["input_ids"] + response["input_ids"]
    # print(input_ids)
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    # print(attention_mask)
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    # print(labels)
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [6]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
print(tokenized_ds)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 48818
})


In [7]:
tokenizer.decode(tokenized_ds[0]["input_ids"])

'Human:保持健康的三个提示。\n\nAssistant: 以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。</s>'

In [8]:
tokenizer.decode(list(filter(lambda x : x != -100, tokenized_ds[0]["labels"])))

'以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。</s>'

# 4.创建模型

In [9]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh")
# model

In [10]:
sum(param.numel() for param in model.parameters())

1303111680

# Prompt_Tuning

## 1.配置文件

In [11]:
from peft import PromptTuningConfig, get_peft_model, TaskType, PromptTuningInit

# soft rompt
config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    num_virtual_tokens=10
)
config

PromptTuningConfig(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, peft_type=<PeftType.PROMPT_TUNING: 'PROMPT_TUNING'>, auto_mapping=None, peft_version='0.18.0', base_model_name_or_path=None, revision=None, inference_mode=False, num_virtual_tokens=10, token_dim=None, num_transformer_submodules=None, num_attention_heads=None, num_layers=None, modules_to_save=None, prompt_tuning_init=<PromptTuningInit.RANDOM: 'RANDOM'>, prompt_tuning_init_text=None, tokenizer_name_or_path=None, tokenizer_kwargs=None)

## 2.创建模型

In [12]:
model = get_peft_model(model, config)
model

PeftModelForCausalLM(
  (base_model): BloomForCausalLM(
    (transformer): BloomModel(
      (word_embeddings): Embedding(46145, 2048)
      (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (h): ModuleList(
        (0-23): 24 x BloomBlock(
          (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (self_attention): BloomAttention(
            (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
            (dense): Linear(in_features=2048, out_features=2048, bias=True)
            (attention_dropout): Dropout(p=0.0, inplace=False)
          )
          (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (mlp): BloomMLP(
            (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
            (gelu_impl): BloomGelu()
            (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
          )
        )
      )

In [13]:
model.print_trainable_parameters()

trainable params: 20,480 || all params: 1,303,132,160 || trainable%: 0.0016


# 5.配置训练参数

In [14]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1,
    save_steps=20,
)

In [15]:
import torch
import gc

# 释放所有未使用的CUDA缓存
torch.cuda.empty_cache()
# 垃圾回收，清理未引用的张量
gc.collect()

# 验证显存占用（可选，查看是否释放成功）
print("GPU显存占用情况：")
print(f"已用显存: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
print(f"剩余显存: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")

GPU显存占用情况：
已用显存: 0.00 MB
剩余显存: 0.00 MB


# 6.创建模型

In [16]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer,padding=True)
)

C:\Users\19237\AppData\Local\Temp\ipykernel_25788\4201552055.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


# 7.训练

In [17]:
trainer.train()

Step,Training Loss



KeyboardInterrupt



# 加载训练好的PEFT模型

In [ ]:
from peft import PeftModel
peft_model = PeftModel.from_pretrained(model=model, model_id="")

In [ ]:
# 推理部分
peft_model = peft_model.cuda() # 推理放在cpu上
ipt = tokenizer("Human: {}\n{}".format("考试有那些技巧","").strip() + "\n\nAssistant: ", return_tensors="pt").to(peft_model.device)
print(tokenizer.decode(peft_model.generate(**ipt, max_new_tokens=128,do_sample=True)[0], skip_special_tokens=True))

# 8.模型推理

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation",model=model, tokenizer=tokenizer, device=0)

In [ ]:
ipt = "Human: {}\n{}".format("考试有那些技巧","").strip() + "\n\nAssistant: "
pipe(ipt,max_length=256)